# Modules

In [ ]:
# Basics
import pandas as pd
import numpy as np
import json
import logging
import os
import pickle
from IPython.display import display
import time

## Neo4j
from flask import (Flask, g, request, Response)
from neo4j import (GraphDatabase, basic_auth)

## Spreadsheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials

## BigQuery
from google.cloud import bigquery

## recommendation
import surprise
from sklearn.model_selection import train_test_split
from surprise import Dataset
from surprise import Reader
from surprise import SVD

pd.set_option("display.max_columns", None)

ModuleNotFoundError: ignored

In [ ]:
## prepare the local DBMS
# 아래 코드는 비밀번호말고 전혀 수정할 내용 없음
scheme = "bolt" 
host_name = "localhost"
port = 7687
url = f"{scheme}://{host_name}:{port}"
user = "neo4j"

# 지금 내가 실행한 DB의 비밀번호를 입력해주자.
password = "0000"

driver = GraphDatabase.driver(url, auth=(user, password))
# driver.verify_connectivity()

driver.session(database="neo4j") # 지금 start시킨 database 이름

In [ ]:
!export GOOGLE_APPLICATION_CREDENTIALS = '/Users/sungwoo/Desktop/Computer/bkms/bkms-bq-956d2fdf3af7.json'

## 여기에도 json 파일의 주소를 붙여넣습니다.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/sungwoo/Desktop/Computer/bkms/bkms-bq-956d2fdf3af7.json"
client = bigquery.Client()
project = client.project

zsh:1: bad assignment


In [ ]:
## Import crawling data
df = pd.read_csv("./ratings.csv", engine="python")
id_info = pd.read_csv("./id_info.csv", engine="python")

# New input

In [ ]:
# 건드릴 것 없음
scope = ['https://spreadsheets.google.com/feeds']
# json_file_name은 json파일이 저장된 경로를 잘 식별해주면 됩니다. 
json_file_name = '/Users/sungwoo/Desktop/Computer/bkms/bkms-bq-956d2fdf3af7.json'
# 건드릴 것 없음
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
# 건드릴 것 없음
gc = gspread.authorize(credentials)
# 설문조사 2 url은 아래의 url과 같습니다.
# https://forms.gle/bPf7Cmys3DcPE9gt6
spreadsheet_url = '
'

In [ ]:
doc = gc.open_by_url(spreadsheet_url)
# 시트 불러오기 : 기본적으로 설문지 응답이 자동으로 스프레드 시트에 기록되는 구조라서, 시트 이름도 그에 맞게 자동 설정되었습니다.
worksheet = doc.worksheet('Form Responses 1')
input_ratings = pd.DataFrame.from_dict(worksheet.get_all_records()).iloc[:,1:]

In [ ]:
columns = ["user_id"]
for c in input_ratings.columns.values[1:]:
    columns.append(c.split('[')[-1][:-1])

input_ratings.columns = columns
display(input_ratings)

,user_id,Inception,1917,The Batman,Minari,Land of Mine,It
0,sungwoo31,10,8,8,7,7,6
1,gsdszzang,7,5,3,7,10,5
2,gsdszzang2,3,5,7,8,4,10
3,gsdszzang3,8,10,6,5,8,4


## Load & Recommendation

In [ ]:
titles = input_ratings.columns.values[1:]
user_id = input_ratings.index[-1]
user = input_ratings["user_id"].unique()[user_id]

## aggregate input information
ratings_t = pd.DataFrame(columns=["watcha_movie_id", "title", "user_id", "rating", "significants", "cast"])
for title in titles:
    subdf = id_info[(id_info["title"] == title)]
    ratings_t = ratings_t.append({"watcha_movie_id": subdf["watcha_movie_id"].iloc[0], "title": title,
                                 "user_id": user, "rating": float(input_ratings[title].iloc[user_id]) / 2,
                                 "significants": subdf["significants"].iloc[0], "cast": subdf["cast"].iloc[0]},
                                ignore_index=True)

## Load to Neo4j
# create user node
with driver.session(database="neo4j") as session:
    def create_user(tx, movie_watchaID, user_watchaID, like_movie):
        query = (
            "MATCH (m1:Movie { id: $movie_watchaID }) "
            "MERGE (u1:User { id: $user_watchaID }) "
            "CREATE (u1)-[:LIKE {Likeness: $like_movie}]->(m1) "
            "RETURN m1, u1"
        )
        result = tx.run(query, movie_watchaID=movie_watchaID, user_watchaID=user_watchaID, like_movie=like_movie)
        return [{"m1": record["m1"]["movie_title"]} for record in result]
    
    for i in range(ratings_t.shape[0]):
        movie_watchaID = ratings_t["watcha_movie_id"].iloc[i]
        user_watchaID = ratings_t["user_id"].iloc[i]
        like_movie = ratings_t["rating"].iloc[i]
        result = session.write_transaction(create_user, movie_watchaID=movie_watchaID, 
                                           user_watchaID=user_watchaID, like_movie=like_movie)
        
# create relationship to actors
with driver.session(database="neo4j") as session:
    def create_likeactor(tx, user_watchaID, actor_name, like_actor):
        query = (
            "MATCH (u1:User { id: $user_watchaID }) "
            "MATCH (a1:Actor { name: $actor_name }) "
            "CREATE (u1)-[:LIKE {Likeness: $like_actor}]->(a1) "
            "RETURN a1, u1"
        )
        result = tx.run(query, user_watchaID=user_watchaID, actor_name=actor_name, like_actor=like_actor)
        return [{"a1": record["a1"]["actor_name"]} for record in result]
      
    cast_rating = {}
    for title in titles:
        subdf = ratings_t[(ratings_t["user_id"] == user) & (ratings_t["title"] == title)]
        if subdf.shape[0] != 0:
            weight = subdf["rating"].iloc[0]
            casts = eval(subdf["cast"].iloc[0])
            for actor in casts:
                if actor not in cast_rating:
                    cast_rating[actor] = 0
                if actor in cast_rating:
                    cast_rating[actor] += int(weight)

    for actor in cast_rating:
        cast_rating[actor] = str(int((cast_rating[actor] * 100 / 5)))

    for actor in cast_rating:
        result = session.write_transaction(create_likeactor, user_watchaID=user, actor_name=actor, like_actor=cast_rating[actor])
            
# create relationship to directors
with driver.session(database="neo4j") as session:
    def create_likedirector(tx, user_watchaID, director_name, like_director):
        query = (
            "MATCH (u1:User { id: $user_watchaID }) "
            "MATCH (d1:Director { name: $director_name }) "
            "CREATE (u1)-[:LIKE {Likeness: $like_director}]->(d1) "
            "RETURN d1, u1"
        )
        result = tx.run(query, user_watchaID=user_watchaID, director_name=director_name, like_director=like_director)
        return [{"d1": record["d1"]["director_name"]} for record in result]
    

    director_rating = {}
    for title in titles:
        subdf = ratings_t[(ratings_t["user_id"] == user) & (ratings_t["title"] == title)]
        if subdf.shape[0] != 0:
            weight = subdf["rating"].iloc[0]
            significants = eval(subdf["significants"].iloc[0])
            for director in significants:
                if director not in director_rating:
                    director_rating[director] = 0
                if director in director_rating:
                    director_rating[director] += int(weight)

    for director in director_rating:
        director_rating[director] = str(int((director_rating[director] * 100 / 5)))


    for director in director_rating:
        result = session.write_transaction(create_likedirector, user_watchaID=user, director_name=director, like_director=director_rating[director])
        
## Load to BigQuery
dataset_id = [item.dataset_id for item in list(client.list_datasets())][0]
table_name = [item.table_id for item in client.list_tables(dataset=f"{project}.{dataset_id}")][0]
table_id = f"{project}.{dataset_id}.{table_name}"

ratings = ratings_t[["watcha_movie_id", "title", "user_id", "rating"]]
news_to_insert = []
for i in range(ratings_t.shape[0]):
    row = {col:str(ratings[col].iloc[i]) for col in ratings.columns.values}
    news_to_insert.append(row)
    
json_obj = json.dumps(news_to_insert)

## insert rows
errors = client.insert_rows_json(table_id, news_to_insert)  # Make an API request.
if errors == []:
    print("New rows have been added.")
else:
    print("Encountered errors while inserting rows: {}".format(errors))

New rows have been added.


In [ ]:
### Neo4j recommendation
## 유저가 좋아하는 영화에 출연한 배우가 출연한 다른 영화
recommend_actor_list = []
with driver.session(database="neo4j") as session:
    def recommend_actor(tx, user_id):
        query = (
            "MATCH (u:User{id: $user_id})-[l:LIKE]->(m:Movie)<-[:PLAY]-(a:Actor)-[:PLAY]->(m2:Movie) "
            "where l.Likeness >= 3.5 "
            "return m2.title AS title"
        )
        result = tx.run(query, user_id=user_id)
        return [record["title"] for record in result]
        
    result = session.read_transaction(recommend_actor, user_id=user)
    for record in result:
        recommend_actor_list.append(record)
        if len(recommend_actor_list) == 10:
            break
        
## 유저가 좋아하는 영화를 연출한 감독이 연출한 다른 영화
recommend_director_list = []
with driver.session(database="neo4j") as session:
    def recommend_director(tx, user_id):
        query = (
            "MATCH (u:User{id: $user_id})-[l:LIKE]->(m:Movie)<-[:DIRECT]-(d:Director)-[:DIRECT]->(m2:Movie) "
            "where l.Likeness >= 3.5 "
            "return m2.title AS title"
        )
        result = tx.run(query, user_id=user_id)
        return [record["title"] for record in result]
        
    result = session.read_transaction(recommend_director, user_id=user)
    for record in result:
        recommend_director_list.append(record)
        if len(recommend_director_list) == 10:
            break
        
final_result = list(set(recommend_actor_list + recommend_director_list))
print(f"좋아하는 감독 및 영화 기반 영화 추천은 다음과 같습니다:\n{final_result[:10]}")

좋아하는 감독 및 영화 기반 영화 추천은 다음과 같습니다:
['The Power of the Dog', 'Public Enemies', 'The Last Samurai', 'Road to Perdition', 'Midnight in Paris', 'Sniper: Legacy', 'Project Power', 'G.I. Joe: The Rise of Cobra', 'The Trial of the Chicago 7', 'Smart People']


In [ ]:
## BigQuery recommendation
QUERY = (
    f"SELECT title, user_id, rating FROM `{table_id}` "
)

query_job = client.query(QUERY)
rows = query_job.result()
result_df = rows.to_dataframe()
result_df = result_df[["user_id", "title", "rating"]]

reader = Reader()
unseen = np.setdiff1d(result_df["title"].unique(), ratings_t["title"].unique())

train = Dataset.load_from_df(result_df, reader).build_full_trainset()
test_ = pd.DataFrame({"user_id": user, "title": unseen, "rating": 0})
test = list(test_.itertuples(index=False, name=None))
predictions = SVD().fit(train).test(test)

result = pd.DataFrame(predictions, columns=['user_id', 'title', 'Real_Rating', 'Estimated_Rating', 'details'])
result = result[["title", "Estimated_Rating"]]
# display(result)

# # 평점 가장 높은 10개를 추천한다고 가정
top = result.sort_values(by="Estimated_Rating", ascending=False).reset_index(drop=True)
top10 = top.head(10)
names = list(top10["title"])
display(top10)
print(names)

,title,Estimated_Rating
0,Moneyball,4.273355
1,Phantom Thread,4.273074
2,About Time,4.271817
3,The Irishman,4.263443
4,Midnight in Paris,4.066936
5,Hell or High Water,4.036997
6,Big Fish,4.018791
7,Rise of the Guardians,4.008617
8,Miles Davis: Birth of the Cool,4.008381
9,Ali Wong: Don Wong,3.967587


['Moneyball', 'Phantom Thread', 'About Time', 'The Irishman', 'Midnight in Paris', 'Hell or High Water', 'Big Fish', 'Rise of the Guardians', 'Miles Davis: Birth of the Cool', 'Ali Wong: Don Wong']
